In [2]:
# 시도명 매핑 딕셔너리
sido_map = {
    '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
    '광주': '광주광역시', '울산': '울산광역시', '인천': '인천광역시',
    '대전': '대전광역시',
    
    # 도 단위 및 기타 지역 매핑
    '북춘천': '강원도', '북강릉': '강원도', '울릉도': '경상북도',
    '안동': '경상북도', '포항': '경상북도', '창원': '경상남도',
    '수원': '경기도', '청주': '충청북도', '전주': '전라북도',
    '목포': '전라남도', '여수': '전라남도', '흑산도': '전라남도',
    '홍성': '충청남도', '백령도': '인천광역시', '서귀포': '제주도',
    '제주': '제주도',
    
    # 기타 추가
    '속초': '강원도', '철원': '강원도', '동두천': '경기도', '파주': '경기도',
    '대관령': '강원도', '춘천': '강원도', '강릉': '강원도', '동해': '강원도',
    '원주': '강원도', '영월': '강원도', '충주': '충청북도', '서산': '충청남도',
    '울진': '경상북도', '추풍령': '경상북도', '상주': '경상북도',
    '군산': '전라북도', '통영': '경상남도', '완도': '전라남도', '고창': '전라북도',
    '순천': '전라남도', '진도(첨찰산)': '전라남도', '서청주': '충청북도',
    '고산': '제주도', '성산': '제주도', '진주': '경상남도',
    '강화': '인천광역시', '양평': '경기도', '이천': '경기도',
    '인제': '강원도', '홍천': '강원도', '태백': '강원도', '정선군': '강원도',
    '제천': '충청북도', '보은': '충청북도', '천안': '충청남도',
    '보령': '충청남도', '부여': '충청남도', '금산': '충청남도',
    '세종': '세종특별자치시', '부안': '전라북도', '임실': '전라북도',
    '정읍': '전라북도', '남원': '전라북도', '장수': '전라북도',
    '고창군': '전라북도', '영광군': '전라남도', '김해시': '경상남도',
    '순창군': '전라북도', '북창원': '경상남도', '양산시': '경상남도',
    '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',
    '해남': '전라남도', '고흥': '전라남도', '의령군': '경상남도',
    '함양군': '경상남도', '광양시': '전라남도', '진도군': '전라남도',
    '봉화': '경상북도', '영주': '경상북도', '문경': '경상북도',
    '청송군': '경상북도', '영덕': '경상북도', '의성': '경상북도',
    '구미': '경상북도', '영천': '경상북도', '경주시': '경상북도',
    '거창': '경상남도', '합천': '경상남도', '밀양': '경상남도',
    '산청': '경상남도', '거제': '경상남도', '남해': '경상남도',
    '북부산': '부산광역시'
}

In [3]:
import pandas as pd

# 로드
df_daylight_hour = pd.read_csv("./data/월별_기온_강수량_일조시간_15~24.csv", encoding="cp949")

# 시도명 매핑 적용
df_daylight_hour['시도명'] = df_daylight_hour['지점명'].map(sido_map)

# 날짜 필터링 (생육기) - 3월 15일 ~ 9월 10일로 잡음
df_daylight_hour['일시'] = pd.to_datetime(df_daylight_hour['일시'], format='%Y-%m')
start_date = pd.to_datetime('2000-03-15')
end_date = pd.to_datetime('2000-09-10')
df_daylight_hour = df_daylight_hour[
    df_daylight_hour['일시'].apply( lambda x: start_date.replace(year=x.year) <= x <= end_date.replace(year=x.year) )
]

# 연도, 시도별 일조시간 합계 평균 계산
df_daylight_hour['연도'] = df_daylight_hour['일시'].dt.year
df_daylight_hour_grouped = df_daylight_hour.groupby(['연도', '시도명'])['합계 일조시간(hr)'].sum().reset_index()
df_daylight_hour_grouped['관측소 수'] = df_daylight_hour.groupby(['연도', '시도명'])['지점명'].nunique().values
df_daylight_hour_grouped['평균 일조시간(시간)'] = df_daylight_hour_grouped['합계 일조시간(hr)'] / df_daylight_hour_grouped['관측소 수']
df_daylight_hour_grouped['평균 일조시간(시간)'] = df_daylight_hour_grouped['평균 일조시간(시간)'].round(0).astype(int)
df_daylight_hour_grouped

,연도,시도명,합계 일조시간(hr),관측소 수,평균 일조시간(시간)
0,2015,강원도,16790.1,13,1292
1,2015,경기도,6694.0,5,1339
2,2015,경상남도,15517.2,14,1108
3,2015,경상북도,18112.6,15,1208
4,2015,광주광역시,1168.3,1,1168
...,...,...,...,...,...
161,2024,전라남도,16066.5,13,1236
162,2024,전라북도,12038.7,10,1204
163,2024,제주도,4548.5,4,1137
164,2024,충청남도,7723.8,6,1287


In [4]:
df_daylight_hour_grouped = df_daylight_hour_grouped[df_daylight_hour_grouped['연도'] >= 2018].reset_index(drop=True)
df_daylight_hour_grouped

,연도,시도명,합계 일조시간(hr),관측소 수,평균 일조시간(시간)
0,2018,강원도,17493.0,14,1250
1,2018,경기도,6692.2,5,1338
2,2018,경상남도,16597.1,14,1186
3,2018,경상북도,18771.6,15,1251
4,2018,광주광역시,1234.7,1,1235
...,...,...,...,...,...
113,2024,전라남도,16066.5,13,1236
114,2024,전라북도,12038.7,10,1204
115,2024,제주도,4548.5,4,1137
116,2024,충청남도,7723.8,6,1287


In [6]:
df_daylight_hour_grouped[df_daylight_hour_grouped['연도'] == 2018] # 세종시 데이터 없음
df_daylight_hour_grouped[df_daylight_hour_grouped['시도명'] == '세종특별자치시'] # 2019~2024 데이터만 존재

sejong = {
    '연도': 2018,
    '시도명': '세종특별자치시',
    '합계 일조시간(hr)': df_daylight_hour_grouped[
        (df_daylight_hour_grouped['연도'] == 2018) & 
        (df_daylight_hour_grouped['시도명'] == '대전광역시')
    ]['합계 일조시간(hr)'].values[0],
    '관측소 수': 1,
    '평균 일조시간(시간)': df_daylight_hour_grouped[
        (df_daylight_hour_grouped['연도'] == 2018) & 
        (df_daylight_hour_grouped['시도명'] == '대전광역시')
    ]['평균 일조시간(시간)'].values[0]
}

df_daylight_hour_grouped = pd.concat([df_daylight_hour_grouped, pd.DataFrame([sejong])], ignore_index=True)
df_daylight_hour_grouped.sort_values(by=['연도', '시도명'], inplace=True)
df_daylight_hour_grouped.reset_index(drop=True, inplace=True)
df_daylight_hour_grouped[df_daylight_hour_grouped['시도명'] == '세종특별자치시']

,연도,시도명,합계 일조시간(hr),관측소 수,평균 일조시간(시간)
9,2018,세종특별자치시,1364.7,1,1365
26,2019,세종특별자치시,728.8,1,729
27,2019,세종특별자치시,1364.7,1,1365
44,2020,세종특별자치시,1112.4,1,1112
61,2021,세종특별자치시,1147.9,1,1148
78,2022,세종특별자치시,1214.9,1,1215
95,2023,세종특별자치시,1072.6,1,1073
112,2024,세종특별자치시,1241.7,1,1242


In [7]:
df_daylight_hour_grouped.to_csv("./data/생육기_합계일조시간.csv", index=False, encoding="utf-8")